# Turing Machines

[On Computable Numbers, with an Application to the Entscheidungsproblem](https://people.math.ethz.ch/~halorenz/4students/Literatur/TuringFullText.pdf)  
Proceedings of the London Mathematical Society Series 2, 42, 230-265  
Turing, A. M. (1937)  

[Introduction to the Theory of Computation, Third Edition](https://www.cengage.uk/c/new-edition/9780357670583/)  
Michael Sipser  
Cengage Learning

## Long Multiplication

Multiply 123 by 45.  
You need your times tables for 0 to 9.  
They are a partial list of instructions.  

We would write:

```
 123
x 45
----
 615
4920
----
5535
```

The problem is written as:

```
123x45=
```

You can, with some effort and notation avoid using a 2D grid:

```
123x45=615,4920=5535
```

## Features

- Series of cells called the *tape*, each of which can hold a symbol. Infinite it both directions.
- Special symbol called the *blank* symbol, which means a cell is effectively empty.  
- Read/Write head that can read the symbol in the current cell and overwrite it.
- Read/Write head can move one cell to the left or right at a time.
- Set of *states* that the machine can be in.
- Programming means choosing states, symbols, and rules to perform a given function.

## Definition

https://en.wikipedia.org/wiki/Turing_machine#Formal_definition

A machine is a 7-tuple (Q, T, b, A, d, s, F) where:
- Q is a finite, non-empty set. We call the elements *states*.
- T is a finite, non-empty set called the *tape alphabet*. Any cell can hold one of these.
- b is the *blank symbol*.
- A is the *input alphabet*, which is a subset of T excluding b.
- d is the *transition function* d:(Q\F)xT -> QxTx{L,R}. These are the rules of the machine.
- s is the *initial state*. It must be an element of $Q$.
- F is a subset of Q. The elements are called *final states*.

## Example

Note simulating a Turing machine using Python is weird.  
You have to be careful not rely on any particular feature of Python itself.  

In [1]:
# States.
Q = {'U', 'V', 'A', 'F'}

# Alphabet - underscore is b.
T = {'0', '1', '_'} 

# Blank symbol.
b = '_'

# Input alphabet.
A = {'0', '1'}

# Transition function.
def d(state, symbol):
    table = {
        ('U', '0'): ('U', '0', 'R'),
        ('U', '1'): ('V', '1', 'R'),
        ('U', '_'): ('A', '_', 'L'),
        ('V', '0'): ('V', '0', 'R'),
        ('V', '1'): ('U', '1', 'R'),
        ('V', '_'): ('F', '_', 'L'),
    }
    return table[(state, symbol)]

# Initial state.
s = 'U'

# Final states.
F = {'A', 'F'}


## Simulator

- Represent the tape with a list.
- Keep track of our position in the string.
- Turing machine tapes have no cell identifiers.
- We only know which cell we are over and how to move one cell left or right.

In [2]:
# Example input.
initial_input = '11101010'

# Example state table.
state_table = {
    ('U', '0'): ('U', '0', 'R'),
    ('U', '1'): ('V', '1', 'R'),
    ('U', '_'): ('A', '_', 'L'),
    ('V', '0'): ('V', '0', 'R'),
    ('V', '1'): ('U', '1', 'R'),
    ('V', '_'): ('F', '_', 'L'),
}


In [3]:
# Find start state from table, convention.
list(state_table.keys())[0][0]

'U'

In [4]:
# Find final states from table, convention.
set([i[0] for i in state_table.values()]) - set(i[0] for i in state_table.keys()) 

{'A', 'F'}

In [8]:
def simulate(table, initial_input, verbose=True):
    """Simulate a Turing machine."""
    if verbose is True:
        print(f'Length of input: {len(initial_input)}')
    # Keep track of number of operations.
    no_ops = 0

    # Check for empty input.
    if initial_input == '':
        initial_input = '_'
    # Make input a list.
    tape = list(initial_input)
    # Initial position on the tape, convention.
    pos = 0

    # Initial state - note dictionaries preserve insertion.
    state = list(state_table.keys())[0][0]
    # All states in first column.
    states_1 = set(i[0] for i in state_table.keys())
    # All states in third column.
    states_3 = set([i[0] for i in state_table.values()])
    # Final states.
    F = states_3 - states_1

    if verbose is True:
        # Print starting config.
        print(state + ''.join(tape))

    # While not in a final state.
    while state not in F:
        # Get current symbol.
        symbol = tape[pos]
        # Get next state, symbol, and direction.
        state, tape[pos], direction = table[(state, symbol)]
        # Increment number of operations.
        no_ops += 1
        # Move cell.
        if direction == 'L':
            pos -= 1
        else:
            pos += 1
        # Fix problems with Python lists being finite
        if pos < 0:
            tape = [b] + tape
            pos = 0
        if pos == len(tape):
            tape.append('_')

        if verbose is True:
            # Print current configuration.
            print(''.join(tape[:pos]) + state + ''.join(tape[pos:]))

    if verbose is True:
        # Print number of operations.
        print(f'Number of operations: {no_ops}')
    
    # Return final state.
    return state

In [9]:
simulate(state_table, initial_input)

Length of input: 8
U11101010
1V1101010
11U101010
111V01010
1110V1010
11101U010
111010U10
1110101V0
11101010V_
1110101F0_
Number of operations: 9


'F'

In [10]:
# Empty input.
simulate(state_table, "")

Length of input: 0
U_
A__
Number of operations: 1


'A'

In [ ]:
for eg in ['', '0', '100', '10101011101100101']:
    simulate(state_table, eg)

Length of input: 0
U_
A__
Number of operations: 1
Length of input: 1
U0
0U_
A0_
Number of operations: 2
Length of input: 3
U100
1V00
10V0
100V_
10F0_
Number of operations: 4
Length of input: 17
U10101011101100101
1V0101011101100101
10V101011101100101
101U01011101100101
1010U1011101100101
10101V011101100101
101010V11101100101
1010101U1101100101
10101011V101100101
101010111U01100101
1010101110U1100101
10101011101V100101
101010111011U00101
1010101110110U0101
10101011101100U101
101010111011001V01
1010101110110010V1
10101011101100101U_
1010101110110010A1_
Number of operations: 18


## Zeros and Ones

$L = \{\epsilon, 01, 0011, 000111, \ldots \}$

In [12]:
# Example state table.
state_table = {
    ('U', '0'): ('V', '_', 'R'),
    ('U', '1'): ('F', '1', 'L'),
    ('U', '_'): ('A', '_', 'L'),
    ('V', '0'): ('V', '0', 'R'),
    ('V', '1'): ('V', '1', 'R'),
    ('V', '_'): ('W', '_', 'L'),
    ('W', '0'): ('F', '0', 'L'),
    ('W', '1'): ('X', '_', 'L'),
    ('W', '_'): ('F', '_', 'L'),
    ('X', '0'): ('X', '0', 'L'),
    ('X', '1'): ('X', '1', 'L'),
    ('X', '_'): ('U', '_', 'R'),
}


In [13]:
simulate(state_table, '0011')

Length of input: 4
U0011
_V011
_0V11
_01V1
_011V_
_01W1_
_0X1__
_X01__
X_01__
_U01__
__V1__
__1V__
__W1__
_X____
__U___
_A____
Number of operations: 15


'A'

## End